<ul class="breadcrumb">
  <li><a href="1_Conventional_Linear_Mixed_Model.ipynb">Bayesian Linear Mixed Models (Conventional)</a></li>
  <li><a href="2_Linear_Additive_Genetic_Model.ipynb">Bayesian Linear Additive Genetic Model</a></li> 
  <li><a href="3_Genomic_Linear_Mixed_Model.ipynb">Bayesian Linear Mixed Models (Genomic Data)</a></li>
</ul>

<div class="span5 alert alert-success">
 <font size="5" face="Georgia">Single-step Bayesian Regression (Incomplete Genomic Data)</font> 
</div>

<button type="button" class="btn btn-lg btn-primary">Step 1: Load Packages</button> 

In [1]:
using JWAS,JWAS.Datasets,DataFrames,CSV, LinearAlgebra

<button type="button" class="btn btn-lg btn-primary">Step 2: Read data</button> 

In [2]:
phenofile  = Datasets.dataset("example","phenotypes_ssbr.txt")
pedfile    = Datasets.dataset("example","pedigree.txt")
genofile   = Datasets.dataset("example","genotypes.txt")

phenotypes = CSV.read(phenofile,delim = ',',header=true)
pedigree   = get_pedigree(pedfile,separator=",",header=true);

coding pedigree... 100%|████████████████████████████████| Time: 0:00:00


Finished!


In [3]:
head(phenotypes)

,ID,y1,y2,y3,x1,x2,x3,dam
,String⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Int64⍰,String⍰,String⍰
1,a1,-0.06,3.58,-1.18,0.9,2,m,0
2,a2,-0.6,4.9,0.88,0.3,1,f,0
3,a3,-2.07,3.19,0.73,0.7,2,f,0
4,a4,-2.63,6.97,-0.83,0.6,1,m,a2
5,a5,2.31,3.5,-1.52,0.4,2,m,a2
6,a6,0.93,4.87,-0.01,5.0,2,f,a3


<div class="span5 alert alert-success">
 <font size="5" face="Georgia">Single-trait Single-step Bayesian Regression (Incomplete Genomic Data)</font> 
</div>

<button type="button" class="btn btn-lg btn-primary">Step 3: Build Model Equations</button> 

In [4]:
model_equation1  ="y1 = intercept + x1*x3 + x2 + x3 + ID + dam";

In [5]:
R      = 1.0
model1 = build_model(model_equation1,R);

<button type="button" class="btn btn-lg btn-primary">Step 4: Set Factors or Covariates</button> 

In [6]:
set_covariate(model1,"x1");

<button type="button" class="btn btn-lg btn-primary">Step 5: Set Random or Fixed Effects</button> 

In [7]:
G1 = 1.0
G2 = [1.0 0.5
      0.5 1.0]
set_random(model1,"x2",G1);
set_random(model1,"ID dam",pedigree,G2);

<button type="button" class="btn btn-lg btn-primary">Step 6: Use Genomic Information</button> 

In [8]:
G3 =1.0
add_genotypes(model1,genofile,G3,separator=',');

┌ Warning: readtable is deprecated, use CSV.read from the CSV package instead
│   caller = ip:0x0
└ @ Core :-1


5 markers on 7 individuals were added.


<button type="button" class="btn btn-lg btn-primary">Step 7: Run Analysis</button> 

In [9]:
outputEBV(model1,["a1","a2","a3"]);
outputMCMCsamples(model1,"x2")
out1=runMCMC(model1,phenotypes,methods="RR-BLUP",single_step_analysis=true,
    pedigree=pedigree,chain_length=5000,output_samples_frequency=100,outputEBV=true);


The prior for marker effects variance is calculated from the genetic variance and π. The mean of the prior for the marker effects variance is: 0.492462


A Linear Mixed Model was build using model equations:

y1 = intercept + x1*x3 + x2 + x3 + ID + dam

Model Information:

Term            C/F          F/R            nLevels
intercept       factor       fixed                1
x1*x3           interaction  fixed                2
x2              factor       random               2
x3              factor       fixed                2
ID              factor       random              12
dam             factor       random              12
ϵ               factor       random               5
J               covariate    fixed                1

MCMC Information:

methods                                     RR-BLUP
chain_length                                   5000
burnin                                            0
estimatePi                                    false
starting_value               

running MCMC for RR-BLUP...100%|████████████████████████| Time: 0:00:01


<button type="button" class="btn btn-lg btn-primary">Check Results</button> 

In [10]:
keys(out1)

Base.KeySet for a Dict{Any,Any} with 6 entries. Keys:
  "Posterior mean of polygenic effects covariance matrix"
  "EBV_y1"
  "Posterior mean of marker effects"
  "Posterior mean of residual variance"
  "Posterior mean of marker effects variance"
  "Posterior mean of location parameters"

In [11]:
out1["EBV_y1"]

3×2 Array{Any,2}:
 "a1"  -0.167823
 "a2"  -0.221442
 "a3"  -1.85712 

In [12]:
out1["Posterior mean of marker effects"]

5×2 Array{Any,2}:
 "m1"  -0.0391276
 "m2"  -0.176596 
 "m3"   0.358624 
 "m4"  -0.241618 
 "m5"  -0.107405 

<div class="span5 alert alert-success">
 <font size="5" face="Georgia">Multi-trait Single-step Bayesian Regression (Incomplete Genomic Data)</font> 
</div>

<button type="button" class="btn btn-lg btn-primary">Step 3: Build Model Equations</button> 

In [13]:
model_equation2 ="y1 = intercept + x1 + x3 + ID + dam
                  y2 = intercept + x1 + x2 + x3 + ID
                  y3 = intercept + x1 + x1*x3 + x2 + ID";

In [14]:
R      = [1.0 0.5 0.5
          0.5 1.0 0.5
          0.5 0.5 1.0]
model2 = build_model(model_equation2,R);

<button type="button" class="btn btn-lg btn-primary">Step 4: Set Factors or Covariates</button> 

In [15]:
set_covariate(model2,"x1");

<button type="button" class="btn btn-lg btn-primary">Step 5: Set Random or Fixed Effects</button> 

In [16]:
G1 = [1.0 0.5
      0.5 1.0]
G2 = [1.0 0.5 0.5 0.5
      0.5 1.0 0.5 0.5
      0.5 0.5 1.0 0.5
      0.5 0.5 0.5 1.0]
set_random(model2,"x2",G1);
set_random(model2,"ID dam",pedigree,G2);

x2 is not found in model equation 1.
dam is not found in model equation 2.
dam is not found in model equation 3.


<button type="button" class="btn btn-lg btn-primary">Step 6: Use Genomic Information</button> 

In [17]:
G3 = [1.0 0.5 0.5
      0.5 1.0 0.5
      0.5 0.5 1.0]
add_genotypes(model2,genofile,G3,separator=',');

5 markers on 7 individuals were added.


<button type="button" class="btn btn-lg btn-primary">Step 7: Run Analysis</button> 

In [18]:
out2=runMCMC(model2,phenotypes,methods="BayesC",estimatePi=true,single_step_analysis=true,pedigree=pedigree,chain_length=5000,output_samples_frequency=100);

Pi (Π) is not provided.
Pi (Π) is generated assuming all markers have effects on all traits.

The prior for marker effects covariance matrix is calculated from genetic covariance matrix and Π. The mean of the prior for the marker effects covariance matrix is: 
 0.492462  0.246231  0.246231
 0.246231  0.492462  0.246231
 0.246231  0.246231  0.492462


A Linear Mixed Model was build using model equations:

y1 = intercept + x1 + x3 + ID + dam
y2 = intercept + x1 + x2 + x3 + ID
y3 = intercept + x1 + x1*x3 + x2 + ID

Model Information:

Term            C/F          F/R            nLevels
intercept       factor       fixed                1
x1              covariate    fixed                1
x3              factor       fixed                2
ID              factor       random              12
dam             factor       random              12
x2              factor       random               2
x1*x3           interaction  fixed                2
ϵ               factor       random           

running MCMC for BayesC...100%|█████████████████████████| Time: 0:00:04


<button type="button" class="btn btn-lg btn-primary">Check Results</button> 

In [19]:
keys(out2)

Base.KeySet for a Dict{Any,Any} with 11 entries. Keys:
  "Model frequency"
  "Posterior mean of marker effects"
  "EBV_y3"
  "Posterior mean of location parameters"
  "EBV_y2"
  "Posterior mean of polygenic effects covariance matrix"
  "EBV_y1"
  "Posterior mean of marker effects covariance matrix"
  "Posterior mean of residual variance"
  "Posterior mean of Pi"
  "Posterior mean of marker effects variance"

In [20]:
out1["Posterior mean of location parameters"]

,Trait,Effect,Level,Estimate
,Any,Any,Any,Any
1,1,intercept,intercept,16.2345
2,1,x1*x3,x1 * m,-2.87112
3,1,x1*x3,x1 * f,0.547077
4,1,x2,2,0.0651677
5,1,x2,1,-0.0626484
6,1,x3,m,-14.5087
7,1,x3,f,-17.1091
8,1,ID,a2,-0.119604
9,1,ID,a12,0.126861
